<a href="https://colab.research.google.com/github/ssnirgudkar/UNet/blob/main/IR_autolabel_7classes_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ww will try to auto label ssnirgudkar/IR-2019-11-05-philios-13-35-55. It has 32 images


In [12]:
# connect to google drive so that we can use the saved model
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [15]:
# although we dont use detectron, segments.ai does for visualization when it uploads the image. So we have to install detectron libraries

import torch
assert torch.cuda.is_available(), 'You need a GPU! In Colab, go to Runtime -> Change runtime type -> Hardware accelerator -> GPU'
!pip install pyyaml==5.1
!pip install torch==1.8.1
!pip install torchvision==0.9.1
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html


     |████████████████████████████████| 274 kB 13.5 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44091 sha256=86a1cf727f0bb0ec57fc36b62b936cac1cf4cf5093a8c7ac0958417c3a65e49b
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


     |████████████████████████████████| 804.1 MB 2.9 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.


     |████████████████████████████████| 17.4 MB 161 kB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.3 MB 827 kB/s 
     |████████████████████████████████| 49 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 19.5 MB/s 
     |████████████████████████████████| 130 kB 24.2 MB/s 
     |████████████████████████████████| 74 kB 3.2 MB/s 
     |████████████████████████████████| 743 kB 26.1 MB/s 
     |████████████████████████████████| 721 kB 47.3 MB/s 
     |████████████████████████████████| 112 kB 48.5 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210722-py3-none-any.whl size=60612 sha256=f1c18f387974a838cbad67a4c4df9cfde54f7595d732b7534a5ff1243a0bba4a
  Stored in directory: /root/.cach

In [ ]:
pip install --upgrade segments-ai

In [ ]:
from segments import SegmentsClient
api_key = "a89182567b17766b91773021b18d04574cd75109"
client = SegmentsClient(api_key)

In [3]:
dataset_identifier = "ssnirgudkar/IR-2019-11-05-philios-13-35-55"

In [4]:
!git clone https://github.com/segments-ai/fast-labeling-workflow/
%cd fast-labeling-workflow


fatal: destination path 'fast-labeling-workflow' already exists and is not an empty directory.
/content/fast-labeling-workflow


In [5]:
dataset_identifier_infer = "ssnirgudkar/IR-2019-11-05-philios-13-35-55"
dataset_infer = client.get_dataset(dataset_identifier_infer)


In [ ]:
from segments import SegmentsDataset
from segments.utils import export_dataset

infer_release_file = client.get_release(dataset_identifier_infer,'V1.0')
infer_releasedataset = SegmentsDataset(infer_release_file, labelset='ground-truth', filter_by=['Unlabeled'])


In [14]:
# create a model from the saved model
#load the saved model 
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img

reconstructed_model = keras.models.load_model("/content/drive/MyDrive/Models/UNet_7classes_1.0")





In [ ]:
from segments import SegmentsDataset
from segments.utils import export_dataset
from segments.utils import bitmap2file

import cv2 
import numpy as np 
import os 
from utils import visualize 
#this needs 3 channel image. so had to convert grayscale IR to RGB - 3 channels by copying the same pixel value on all 3
#add logic to check if the image is 1 channel, and if so convert it to 3 channel
for sample in infer_releasedataset:
#for sample in trial_releasedataset:
   infer_image = sample['image'] 
      
   #imgcv = cv2.imread(infer_image)
   #assert not isinstance(imgcv,type(None)), 'image not found'
   #print ("file read = {0}".format(imgcv))
   #print(os.getcwd())  # prints current working directory
   #infer_image2 = RemoveAlphaChannel[infer_image, White]
   #4channel CYMK image is not taken by detectron. it needs 3 channels. so converting to RGB. we can add a conditional check
   # to first check if it's 4 channels and then convert only if it is
   #infer_image2 = infer_image.convert("RGB")
   Model = reconstructed_model.predict(infer_image)
   segmentation_bitmap, annotations = Model(infer_image)
   print("segmentation bitmap executed")
   #visualize the predictions 
   #visualize(infer_image2, segmentation_bitmap)
   #print(annotations)

# Upload the predictions to Segments.ai so that we can check the effectiveness of auto labels
   file = bitmap2file(segmentation_bitmap)
   asset = client.upload_asset(file, 'label.png')    
   attributes = {
        'format_version': '0.1',
        'annotations': annotations,
        'segmentation_bitmap': { 'url': asset['url'] },
   }
   client.add_label(sample['uuid'], 'ground-truth', attributes, label_status='PRELABELED')